In [1]:
import re
import struct
import binascii
import datetime
from collections import namedtuple

Header = namedtuple('Header', ['data_type', 'controller_id', 'inductive_id', 'num_bytes',
                    'mflm_flag', 'timestamp', 'block', 'crc'])

CTRecord = namedtuple('CTRecord', ['temp', 'cond', 'pressure', 'time', 'inductive_id'])

OFFSET = (datetime.datetime(2000, 1, 1) - datetime.datetime(1970, 1, 1)).total_seconds()

In [2]:
def decode_header(header):
    '''
    Header Description
    Offset    NumBytes    Value/Type    Description
    00        1           [0x01]        SOH, Start of Header
    01        2           [A-Z0-9]      Identifier for Instrument/DataType 
    03        5           [0-9]         ControllerID 
    08        2           [0-9]         Number of Instrument / Inductive ID 
    10        1           [_]           Spacer 5F hex
    11        4           [0-9A-F]      Number of Data Bytes, Hex Number (N bytes) 
    15*       1           [A-z0-9]      MFLM Processing Flag (u for unprocessed)
                                        OR HYPM Decimation Factor (A=1, Z=26) or HYPM d for decimated 
    16        8           [0-9A-F]      POSIX Timestamp of Controller, Hex Number 
    24        1           [_]           Spacer 5F hex
    25        2           [0-9A-F]      Block Number, HEX, 0 .. 255 (overflow) 
    27        1           [_]           Spacer 5F hex
    28        4           [0-9A-F]      CRC Checksum DataBytes CCITT-16 Reversed (0x8408) 
    32        1           [0x02]        STX, Start of Text 
    33        N                         Data Bytes
    33+       1           [0x03]        ETX, End of Text
    '''
    data_type = header[:2]
    controller_id = header[2:7]
    inductive_id = int(header[7:9])
    num_bytes = int(header[10:14], 16)
    mflm_flag = header[14:15]
    timestamp = int(header[15:23], 16)
    block = int(header[24:26], 16)
    crc = int(header[27:31], 16)
    return Header(data_type, controller_id, inductive_id, num_bytes, mflm_flag, timestamp, block, crc)


def decode_ct_records(chunk, inductive_id):
    sample_re = re.compile(r'[A-F0-9]{22}\r\n')
    records = []
    for record in sample_re.findall(chunk):
        temp = int(record[:5], 16)
        cond = int(record[5:10], 16)
        pressure, secs = struct.unpack('<HI', binascii.a2b_hex(record[10:22]))
#         pressure = int(record[12:14] + record[10:12], 16)
#         secs, = struct.unpack('<I', binascii.a2b_hex(record[14:22]))
        time = datetime.datetime.utcfromtimestamp(secs + OFFSET)
        records.append(CTRecord(temp, cond, pressure, time, inductive_id))
    return records
    

In [3]:
running = True
fh = open('CTD15907.DAT')
records = []

while running:
    flag = fh.read(1)
    if flag == '':
        break
    if flag == '\x01':
        # possible header
        header = decode_header(fh.read(31))
        body = fh.read(header.num_bytes)
        if header.data_type == 'CT':
            records.extend(decode_ct_records(body, header.inductive_id))
            
for r in records:
    print r

CTRecord(temp=225645, cond=50003, pressure=2786, time=datetime.datetime(2013, 7, 21, 18, 0, 1), inductive_id=67)
CTRecord(temp=230348, cond=433201, pressure=2854, time=datetime.datetime(2013, 7, 21, 18, 15, 1), inductive_id=67)
CTRecord(temp=190590, cond=398373, pressure=3245, time=datetime.datetime(2013, 7, 21, 18, 30, 1), inductive_id=67)
CTRecord(temp=148912, cond=363598, pressure=4213, time=datetime.datetime(2013, 7, 21, 18, 45, 1), inductive_id=67)
CTRecord(temp=144238, cond=367491, pressure=5167, time=datetime.datetime(2013, 7, 21, 19, 0, 1), inductive_id=67)
CTRecord(temp=138187, cond=364528, pressure=6269, time=datetime.datetime(2013, 7, 21, 19, 15, 1), inductive_id=67)
CTRecord(temp=138108, cond=364538, pressure=6351, time=datetime.datetime(2013, 7, 21, 19, 30, 1), inductive_id=67)
CTRecord(temp=137844, cond=364831, pressure=6839, time=datetime.datetime(2013, 7, 21, 19, 45, 1), inductive_id=67)
CTRecord(temp=144973, cond=361960, pressure=4597, time=datetime.datetime(2013, 7, 2